In [1]:
import os
import h5py
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
os.environ['CUDA_VISIBLE_DEVICES']='0'

tf.enable_eager_execution()
print(tf.__version__)

/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.

1.14.0


In [2]:
num_classes    = 9
batch_size     = 16
train_days     = 3
epochs         = 500
learning_rate  = 0.0001

In [3]:
#Read data
hf = h5py.File('/home/kjakkala/mmwave/data/source_data.h5', 'r')
X_data = np.expand_dims(hf.get('X_data'), axis=-1).astype(np.float32)
y_data = np.array(hf.get('y_data')).astype(np.int32)
classes = list(hf.get('classes'))
hf.close()
print(X_data.shape, y_data.shape, "\n", classes)

(9145, 128, 1024, 1) (9145, 2) 
 [b'arahman3', b'harika', b'hchen32', b'jlaivins', b'kjakkala', b'pjanakar', b'ppinyoan', b'pwang13', b'upattnai', b'wrang']


In [4]:
#balence dataset to 95 samples per day for each person
X_data_tmp = []
y_data_tmp = []
for day in range(10):
  for idx in range(len(classes)):
    X_data_tmp.extend(X_data[(y_data[:, 0] == idx) & (y_data[:, 1] == day)][:95])
    y_data_tmp.extend(y_data[(y_data[:, 0] == idx) & (y_data[:, 1] == day)][:95])
X_data = np.array(X_data_tmp)
y_data = np.array(y_data_tmp)
del X_data_tmp, y_data_tmp
print(X_data.shape, y_data.shape)

(8740, 128, 1024, 1) (8740, 2)


In [5]:
#remove harika's data
X_data = np.delete(X_data, np.where(y_data[:, 0] == 1)[0], 0)
y_data = np.delete(y_data, np.where(y_data[:, 0] == 1)[0], 0)

#update labes to handle 9 classes instead of 10
y_data[y_data[:, 0] >= 2, 0] -= 1
del classes[1]
print(X_data.shape, y_data.shape, "\n", classes)

(8550, 128, 1024, 1) (8550, 2) 
 [b'arahman3', b'hchen32', b'jlaivins', b'kjakkala', b'pjanakar', b'ppinyoan', b'pwang13', b'upattnai', b'wrang']


In [6]:
#split days of data to train and test
X_src = X_data[y_data[:, 1] < train_days]
y_src = y_data[y_data[:, 1] < train_days, 0]
y_src = np.eye(len(classes))[y_src]
X_train_src, X_test_src, y_train_src, y_test_src = train_test_split(X_src,
                                                                    y_src,
                                                                    stratify=y_src,
                                                                    test_size=0.10,
                                                                    random_state=42)

X_trg  = X_data[y_data[:, 1] >= train_days]
y_trg  = y_data[y_data[:, 1] >= train_days, 0]
y_trg = np.eye(len(classes))[y_trg]
X_train_trg, X_test_trg, y_train_trg, y_test_trg = train_test_split(X_trg,
                                                                    y_trg,
                                                                    stratify=y_trg,
                                                                    test_size=0.20,
                                                                    random_state=42)
del X_src, y_src, X_trg, y_trg

y_train_trg = y_train_trg.astype(np.int64)
y_test_trg  = y_test_trg.astype(np.int64)
y_train_src = y_train_src.astype(np.int64)
y_test_src  = y_test_src.astype(np.int64)

print(X_train_src.shape, y_train_src.shape,  X_test_src.shape, y_test_src.shape, X_train_trg.shape, y_train_trg.shape, X_test_trg.shape, y_test_trg.shape)

(2308, 128, 1024, 1) (2308, 9) (257, 128, 1024, 1) (257, 9) (4788, 128, 1024, 1) (4788, 9) (1197, 128, 1024, 1) (1197, 9)


In [7]:
#standardise dataset
src_mean = np.mean(X_train_src)
X_train_src -= src_mean
src_std  = np.std(X_train_src)
X_train_src /= src_std

X_test_src -= src_mean
X_test_src /= src_std

trg_mean = np.mean(X_train_trg)
X_train_trg -= trg_mean
trg_std  = np.std(X_train_trg)
X_train_trg /= trg_std

X_test_trg -= src_mean
X_test_trg /= src_std

print(X_train_src.shape, y_train_src.shape,  X_test_src.shape, y_test_src.shape, X_train_trg.shape, y_train_trg.shape, X_test_trg.shape, y_test_trg.shape)

(2308, 128, 1024, 1) (2308, 9) (257, 128, 1024, 1) (257, 9) (4788, 128, 1024, 1) (4788, 9) (1197, 128, 1024, 1) (1197, 9)


In [8]:
#get tf.data objects for each set
src_train_set = tf.data.Dataset.from_tensor_slices((X_train_src, y_train_src))
src_train_set = src_train_set.shuffle(X_train_src.shape[0])
src_train_set = src_train_set.batch(batch_size, drop_remainder=True)
src_train_set = src_train_set.prefetch(tf.data.experimental.AUTOTUNE)

trg_train_set = tf.data.Dataset.from_tensor_slices((X_train_trg, y_train_trg))
trg_train_set = trg_train_set.shuffle(X_train_trg.shape[0])
trg_train_set = trg_train_set.batch(batch_size, drop_remainder=True)
trg_train_set = trg_train_set.prefetch(tf.data.experimental.AUTOTUNE)
trg_train_set = trg_train_set.repeat(-1)

src_test_set = tf.data.Dataset.from_tensor_slices((X_test_src, y_test_src))
src_test_set = src_test_set.batch(batch_size, drop_remainder=False)
src_test_set = src_test_set.prefetch(tf.data.experimental.AUTOTUNE)

trg_test_set = tf.data.Dataset.from_tensor_slices((X_test_trg, y_test_trg))
trg_test_set = trg_test_set.batch(batch_size, drop_remainder=False)
trg_test_set = trg_test_set.prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
L2_WEIGHT_DECAY = 1e-4
BATCH_NORM_DECAY = 0.9
BATCH_NORM_EPSILON = 1e-5

class IdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters, stage, block):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    super().__init__(name='stage-' + str(stage) + '_block-' + block)

    filters1, filters2, filters3 = filters
    bn_axis = -1

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2a')
    self.bn2a = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2a')

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size,
                                         padding='same',
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2b')
    self.bn2b = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2b')

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2c')
    self.bn2c = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2c')

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.keras.layers.Activation('relu')(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.keras.layers.Activation('relu')(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x = tf.keras.layers.add([x, input_tensor])
    x = tf.keras.layers.Activation('relu')(x)
    return x


"""A block that has a conv layer at shortcut.

Note that from stage 3,
the second conv layer at main path is with strides=(2, 2)
And the shortcut should have strides=(2, 2) as well

Args:
  kernel_size: the kernel size of middle conv layer at main path
  filters: list of integers, the filters of 3 conv layer at main path
  stage: integer, current stage label, used for generating layer names
  block: 'a','b'..., current block label, used for generating layer names
  strides: Strides for the second conv layer in the block.

Returns:
  A Keras model instance for the block.
"""
class ConvBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters, stage, block, strides=(2, 2)):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    super().__init__(name='stage-' + str(stage) + '_block-' + block)

    filters1, filters2, filters3 = filters
    bn_axis = -1

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2a')
    self.bn2a = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2a')

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size,
                                         strides=strides,
                                         padding='same',
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2b')
    self.bn2b = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2b')

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2c')
    self.bn2c = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2c')

    self.conv2s = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         strides=strides,
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '1')
    self.bn2s = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '1')

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.keras.layers.Activation('relu')(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.keras.layers.Activation('relu')(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    shortcut = self.conv2s(input_tensor)
    shortcut = self.bn2s(shortcut, training=training)

    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    return x


"""Instantiates the ResNet50 architecture.

Args:
  num_classes: `int` number of classes for image classification.

Returns:
    A Keras model instance.
"""
class ResNet50(tf.keras.Model):
  def __init__(self, num_classes, num_features):
    super().__init__(name='generator')
    bn_axis = -1

    #self.conv1_pad = tf.keras.layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')
    self.conv1 = tf.keras.layers.Conv2D(32, (7, 7),
                                        strides=(2, 2),
                                        padding='valid',
                                        use_bias=False,
                                        kernel_initializer='he_normal',
                                        kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        name='conv1')
    self.bn1 = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                  momentum=BATCH_NORM_DECAY,
                                                  epsilon=BATCH_NORM_EPSILON,
                                                  name='bn_conv1')
    self.relu1 = tf.keras.layers.Activation('relu', name='relu1')
    self.max_pool1 = tf.keras.layers.MaxPooling2D((3, 3),
                                                  strides=(2, 2),
                                                  padding='same',
                                                  name='max_pool1')

    self.blocks = []
    self.blocks.append(ConvBlock(3, [32, 32, 128], strides=(1, 1), stage=2, block='a'))
    self.blocks.append(IdentityBlock(3, [32, 32, 128], stage=2, block='b'))

    self.blocks.append(ConvBlock(3, [64, 64, 256], stage=3, block='a'))
    self.blocks.append(IdentityBlock(3, [64, 64, 256], stage=3, block='b'))

    self.blocks.append(ConvBlock(3, [64, 64, 256], stage=4, block='a'))
    self.blocks.append(IdentityBlock(3, [64, 64, 256], stage=4, block='b'))

    self.avg_pool = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')
    self.fc1 = tf.keras.layers.Dense(num_features,
                                     activation='relu',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                     kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                     bias_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                     name='fc1')
    self.logits = tf.keras.layers.Dense(num_classes,
                                        activation=None,
                                        kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                        kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        bias_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        name='logits')

  def call(self, img_input, training=False):
    x = self.conv1(img_input)
    x = self.bn1(x, training=training)
    x = self.relu1(x)
    x = self.max_pool1(x)

    for block in self.blocks:
      x = block(x)

    x = self.avg_pool(x)
    fc1 = self.fc1(x)
    logits = self.logits(fc1)
    return logits, fc1

In [10]:
class Discriminator(tf.keras.Model):
  def __init__(self):
    super().__init__(name='discriminator')  
    self.dense1 = tf.keras.layers.Dense(100, activation='relu')
    self.logits = tf.keras.layers.Dense(1, activation=None)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.logits(x)

    return x

In [11]:
def get_cross_entropy_loss(labels, logits):
  loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits)
  return tf.reduce_mean(loss)

def get_domain_confusion_loss(src_logits, trg_logits):
  discriminator_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(src_logits),
                                                               logits=src_logits) + \
                       tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(trg_logits),
                                                               logits=trg_logits)
  return 0.5 * tf.reduce_mean(discriminator_loss)

def virtual_adversarial_images(images, logits, pert_norm_radius=3.5):  
  with tf.GradientTape() as tape:
    # Get normalised noise matrix
    noise = tf.random.normal(shape=tf.shape(images))
    noise = 1e-6 * tf.nn.l2_normalize(noise, axis=tf.range(1, len(noise.shape)))

    # Add noise to image and get new logits
    noise_logits, _ = generator(images + noise, 
                                tf.constant(False, dtype=tf.bool))

    # Get loss from noisey logits
    noise_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=logits, logits=noise_logits)
    noise_loss = tf.reduce_mean(noise_loss)

  # Based on perturbed image loss, get direction of greatest error
  adversarial_noise = tape.gradient(noise_loss, 
                                    [noise],
                                    unconnected_gradients='zero')[0]

  adversarial_noise = tf.nn.l2_normalize(adversarial_noise, 
                                         axis=tf.range(1, 4))

  # return images with adversarial perturbation
  return images + pert_norm_radius * adversarial_noise

In [12]:
train_total_loss         = tf.keras.metrics.Mean(name='train_total_loss')
train_domain_loss        = tf.keras.metrics.Mean(name='train_domain_loss')
train_src_vat_loss       = tf.keras.metrics.Mean(name='train_src_vat_loss')
train_trg_vat_loss       = tf.keras.metrics.Mean(name='train_trg_vat_loss')
train_cond_entropy_loss  = tf.keras.metrics.Mean(name='train_cond_entropy_loss')
train_cross_entropy_loss = tf.keras.metrics.Mean(name='train_cross_entropy_loss')
train_discriminator_loss = tf.keras.metrics.Mean(name='train_discriminator_loss')
src_test_accuracy        = tf.keras.metrics.CategoricalAccuracy(name='src_test_accuracy')
trg_test_accuracy        = tf.keras.metrics.CategoricalAccuracy(name='trg_test_accuracy')
src_train_accuracy       = tf.keras.metrics.CategoricalAccuracy(name='src_train_accuracy')
trg_train_accuracy       = tf.keras.metrics.CategoricalAccuracy(name='trg_train_accuracy')

@tf.function
def train_gen_step(src_images, src_labels, trg_images, trg_labels):  
  with tf.GradientTape() as gen_tape:
    src_logits, src_enc = generator(src_images, training=True)
    trg_logits, trg_enc = generator(trg_images, training=True)  
    src_adver_images    = virtual_adversarial_images(src_images, tf.nn.softmax(src_logits))
    src_adver_logits, _ = generator(tf.stop_gradient(src_adver_images), training=True)
    trg_adver_images    = virtual_adversarial_images(trg_images, tf.nn.softmax(trg_logits))
    trg_adver_logits, _ = generator(tf.stop_gradient(trg_adver_images), training=True)
    src_disc_logits     = discriminator(src_enc)
    trg_disc_logits     = discriminator(trg_enc)

    cross_entropy_loss  = get_cross_entropy_loss(labels=src_labels, 
                                                 logits=src_logits)
    cross_cond_loss     = get_cross_entropy_loss(labels=tf.nn.softmax(trg_logits), 
                                                 logits=trg_logits)
    src_vat_loss        = get_cross_entropy_loss(labels=tf.nn.softmax(tf.stop_gradient(src_logits)),
                                                 logits=src_adver_logits)
    trg_vat_loss        = get_cross_entropy_loss(labels=tf.nn.softmax(tf.stop_gradient(trg_logits)),
                                                 logits=trg_adver_logits)
    domain_loss         = get_domain_confusion_loss(src_logits=trg_disc_logits, 
                                                    trg_logits=src_disc_logits)

    total_loss = 1    * cross_entropy_loss + \
                 1e-2 * domain_loss + \
                 1e-2 * cross_cond_loss + \
                 1e-2 * trg_vat_loss + \
                 1    * src_vat_loss    
    
  gen_gradients = gen_tape.gradient(total_loss, generator.trainable_variables)
  gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))

  src_train_accuracy(src_labels, src_logits)
  trg_train_accuracy(trg_labels, trg_logits)
  train_cross_entropy_loss(cross_entropy_loss)
  train_cond_entropy_loss(cross_cond_loss)
  train_src_vat_loss(src_vat_loss)
  train_trg_vat_loss(trg_vat_loss)
  train_domain_loss(domain_loss)
  train_total_loss(total_loss)
    
@tf.function
def train_disc_step(src_images, trg_images):  
  with tf.GradientTape() as disc_tape:
    _, src_enc          = generator(src_images, training=True)
    _, trg_enc          = generator(trg_images, training=True)  
    src_disc_logits     = discriminator(src_enc)
    trg_disc_logits     = discriminator(trg_enc)    
    domain_conf_loss    = get_domain_confusion_loss(src_logits=src_disc_logits, 
                                                    trg_logits=trg_disc_logits)
  
  disc_gradients = disc_tape.gradient(domain_conf_loss, 
                                      discriminator.trainable_variables)
  disc_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
  train_discriminator_loss(domain_conf_loss)

@tf.function
def test_source_step(source_images, source_labels):
  source_logits, _ = generator(source_images, training=False)
  src_test_accuracy(source_labels, source_logits)

@tf.function
def test_target_step(target_images, target_labels):
  target_logits, _ = generator(target_images, training=False)
  trg_test_accuracy(target_labels, target_logits)

In [13]:
train_template = 'Epoch: {:03d}, TotalL: {:.4f}, CrossE: {:.4f}, CondE: {:.4f}, disc: {:.4f}, domain: {:.4f}, Src VAT: {:.4f}, Trg VAT: {:.4f}, Src Train Acc: {:.2f}, Trg Train Acc: {:.2f}, '
test_template  = 'Src Test Acc: {:.2f}, Trg Test Acc: {:.2f}'

generator      = ResNet50(num_classes, 256)
discriminator  = Discriminator()
disc_optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.5)
gen_optimizer  = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.5)

summary_writer = tf.contrib.summary.create_file_writer('../logs/VADA', flush_millis=10000)
summary_writer.set_as_default()
global_step = tf.train.get_or_create_global_step()

def log_loss():
  with tf.contrib.summary.always_record_summaries():
    tf.contrib.summary.scalar("train_total_loss", train_total_loss.result())
    tf.contrib.summary.scalar("train_cross_entropy_loss", train_cross_entropy_loss.result())
    tf.contrib.summary.scalar("train_cond_entropy_loss", train_cond_entropy_loss.result())
    tf.contrib.summary.scalar("src_train_accuracy", src_train_accuracy.result())
    tf.contrib.summary.scalar("trg_train_accuracy", trg_train_accuracy.result())
    tf.contrib.summary.scalar("train_discriminator_loss", train_discriminator_loss.result())
    tf.contrib.summary.scalar("train_domain_loss", train_domain_loss.result())
    tf.contrib.summary.scalar("train_src_vat_loss", train_src_vat_loss.result())
    tf.contrib.summary.scalar("train_trg_vat_loss", train_trg_vat_loss.result())
    tf.contrib.summary.scalar("src_test_accuracy", src_test_accuracy.result())
    tf.contrib.summary.scalar("trg_test_accuracy", trg_test_accuracy.result())
                                     
for epoch in range(epochs):
  global_step.assign_add(1)  
  for source_data, target_data in zip(src_train_set, trg_train_set):
    train_gen_step(source_data[0], source_data[1], target_data[0], target_data[1])    
    train_disc_step(source_data[0], target_data[0])

  print(train_template.format(epoch+1,
                              train_total_loss.result(),
                              train_cross_entropy_loss.result(),
                              train_cond_entropy_loss.result(),
                              train_discriminator_loss.result(),
                              train_domain_loss.result(),
                              train_src_vat_loss.result(),
                              train_trg_vat_loss.result(),
                              src_train_accuracy.result()*100,
                              trg_train_accuracy.result()*100), end="")

  for target_data in trg_test_set:
    test_target_step(target_data[0], target_data[1])

  for source_data in src_test_set:
    test_source_step(source_data[0], source_data[1])

  print(test_template.format(src_test_accuracy.result()*100,
                             trg_test_accuracy.result()*100))

  log_loss()
  
  train_total_loss.reset_states()
  train_cross_entropy_loss.reset_states()
  train_cond_entropy_loss.reset_states()
  src_train_accuracy.reset_states()
  trg_train_accuracy.reset_states()
  train_discriminator_loss.reset_states()
  train_domain_loss.reset_states()
  train_src_vat_loss.reset_states()
  train_trg_vat_loss.reset_states()
  src_test_accuracy.reset_states()
  trg_test_accuracy.reset_states()

W0820 22:03:28.618605 139835291182848 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0820 22:03:36.934602 139835291182848 deprecation.py:323] From /home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 001, TotalL: 4.4435, CrossE: 2.1988, CondE: 2.1939, disc: 0.6941, domain: 0.6935, Src VAT: 2.1939, Trg VAT: 2.1939, Src Train Acc: 10.24, Trg Train Acc: 11.68, Src Test Acc: 11.28, Trg Test Acc: 17.46
Epoch: 002, TotalL: 4.4001, CrossE: 2.1719, CondE: 2.1776, disc: 0.6938, domain: 0.6942, Src VAT: 2.1777, Trg VAT: 2.1776, Src Train Acc: 13.54, Trg Train Acc: 14.89, Src Test Acc: 25.68, Trg Test Acc: 20.97
Epoch: 003, TotalL: 4.2222, CrossE: 2.0794, CondE: 2.0893, disc: 0.6950, domain: 0.6971, Src VAT: 2.0941, Trg VAT: 2.0893, Src Train Acc: 18.19, Trg Train Acc: 19.18, Src Test Acc: 31.13, Trg Test Acc: 27.49
Epoch: 004, TotalL: 3.9792, CrossE: 1.9542, CondE: 1.9695, disc: 0.6961, domain: 0.6960, Src VAT: 1.9786, Trg VAT: 1.9695, Src Train Acc: 21.35, Trg Train Acc: 21.74, Src Test Acc: 32.68, Trg Test Acc: 27.74


E0820 22:07:40.233280 139835291182848 ultratb.py:155] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kjakkala/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-8ddb00f22bbf>", line 21, in <module>
    train_disc_step(source_data[0], target_data[0])
  File "/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/eager/def_function.py", line 402, in __call__
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/eager/function.py", line 1324, in __call__
    return graph_function._filtered_call(args, kwargs)  # pylint: disable=protected-access
  File "/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/eager/function.py", line 578, in _filtered_call
    (t for t in nest.flatten((args, kwargs), expand_composites=True)
  File "/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/ea

KeyboardInterrupt: 